# Análisis Estadístico - Evolución de Calidad de Código en Estudiantes de Programación Aplicada

**Investigador:** Enel Almonte  
**Institución:** Universidad Católica Nordestana (UCNE)  
**Tesis:** "Análisis de la Evolución de la Calidad del Código en Estudiantes de Programación Aplicada mediante Métricas de SonarCloud"

## Contexto de la Investigación

### Objetivo General
Analizar la evolución de la calidad del código fuente desarrollado por estudiantes de Ingeniería en Sistemas de la UCNE al comparar los proyectos finales de la asignatura Programación Aplicada I y Programación Aplicada II.

### Pregunta de Investigación
¿Cómo evoluciona la calidad del código fuente producido por estudiantes de Ingeniería en Sistemas de la UCNE al transitar de Programación Aplicada I a Programación Aplicada II?

### Hipótesis
- **H₀**: No existe diferencia estadísticamente significativa en las métricas de calidad del código entre AP1 y AP2
- **H₁**: Existe mejora estadísticamente significativa en las métricas de calidad del código de AP1 a AP2

### Variables
- **Variable Independiente**: Intervención pedagógica (taller "Código Limpio" en AP2)
- **Variable Dependiente**: Métricas de calidad del código de SonarCloud

---

## Metodología Estadística
- **Diseño**: Pre-test/Post-test pareado
- **Nivel de significancia**: α = 0.05
- **Corrección**: FDR para comparaciones múltiples
- **Tamaño del efecto**: Cohen's d para interpretación práctica

# 1. Data Loading and Setup
## Importación de Librerías Necesarias

In [6]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import shapiro, wilcoxon, ttest_rel, normaltest
from statsmodels.stats.multitest import multipletests
import warnings
warnings.filterwarnings('ignore')

# Configurar visualizaciones
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Librerías importadas correctamente")
print("📊 Configuración de visualizaciones lista")

✅ Librerías importadas correctamente
📊 Configuración de visualizaciones lista


In [7]:
# Cargar datos desde URL
url = 'https://raw.githubusercontent.com/TesisEnel/Recopilacion_Datos_CalidadCodigo/main/data/Estudiantes_2023-2024_con_metricas_sonarcloud.csv'

try:
    df = pd.read_csv(url)
    print("✅ Datos cargados exitosamente")
    print(f"📊 Dimensiones del dataset: {df.shape}")
    print(f"👥 Número de estudiantes únicos: {df['estudiante_id'].nunique()}")
    print(f"📚 Asignaturas en el dataset: {df['asignatura'].unique()}")
except Exception as e:
    print(f"❌ Error al cargar los datos: {e}")
    print("🔄 Intentando cargar desde archivo local...")
    try:
        df = pd.read_csv('../data/Estudiantes_2023-2024_con_metricas_sonarcloud.csv')
        print("✅ Datos cargados desde archivo local")
    except:
        print("❌ No se pudieron cargar los datos")

✅ Datos cargados exitosamente
📊 Dimensiones del dataset: (60, 41)
❌ Error al cargar los datos: 'estudiante_id'
🔄 Intentando cargar desde archivo local...
✅ Datos cargados desde archivo local


# 2. Exploratory Data Analysis (EDA)
## Análisis Exploratorio de Datos

In [8]:
# Estructura básica del dataset
print("=== ESTRUCTURA DEL DATASET ===")
print(f"Dimensiones: {df.shape}")
print(f"Columnas: {list(df.columns)}")
print("\n=== INFORMACIÓN BÁSICA ===")
print(df.info())
print("\n=== PRIMERAS FILAS ===")
display(df.head())

=== ESTRUCTURA DEL DATASET ===
Dimensiones: (60, 41)
Columnas: ['Id', 'Semestre', 'Estudiante', 'Sexo', 'Email', 'Original_Repo_Ap1', 'Original_Repo_Ap2', 'Sonar_Ap1', 'Sonar_Ap2', 'Sonar_Repo_Ap1', 'Sonar_Repo_Ap2', 'bugs_AP1', 'vulnerabilities_AP1', 'security_hotspots_AP1', 'code_smells_AP1', 'technical_debt_AP1', 'sqale_rating_AP1', 'complexity_AP1', 'cognitive_complexity_AP1', 'coverage_AP1', 'comment_lines_density_AP1', 'duplicated_lines_density_AP1', 'ncloc_AP1', 'reliability_rating_AP1', 'security_rating_AP1', 'open_issues_AP1', 'bugs_AP2', 'vulnerabilities_AP2', 'security_hotspots_AP2', 'code_smells_AP2', 'technical_debt_AP2', 'sqale_rating_AP2', 'complexity_AP2', 'cognitive_complexity_AP2', 'coverage_AP2', 'comment_lines_density_AP2', 'duplicated_lines_density_AP2', 'ncloc_AP2', 'reliability_rating_AP2', 'security_rating_AP2', 'open_issues_AP2']

=== INFORMACIÓN BÁSICA ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 41 columns):
 #

,Id,Semestre,Estudiante,Sexo,Email,Original_Repo_Ap1,Original_Repo_Ap2,Sonar_Ap1,Sonar_Ap2,Sonar_Repo_Ap1,...,sqale_rating_AP2,complexity_AP2,cognitive_complexity_AP2,coverage_AP2,comment_lines_density_AP2,duplicated_lines_density_AP2,ncloc_AP2,reliability_rating_AP2,security_rating_AP2,open_issues_AP2
0,1,2024-01,Aaron Eliezer Hernández García,1,atminifg655@gmail.com,https://github.com/aaron-developer25/SwiftPay.git,https://github.com/aaron-developer25/DealerPOS...,TesisEnel_SwiftPay-Aaron-Ap1,TesisEnel_DealerPOS-Aaron-ap2,https://github.com/TesisEnel/SwiftPay-Aaron-Ap1,...,1.0,1029,1262,NaN,1.0,3.9,14674,3.0,3.0,241
1,2,2023-03,Abraham El Hage Jreij,1,abrahamelhage2003@gmail.com,https://github.com/JPichardo2003/AguaMariaSolu...,https://github.com/A-EHJ/Final_Project_Ap2.git,TesisEnel_AguaMariaSolution-JulioPichardo-ap1,TesisEnel_Final_Project-Abraham-ap2,https://github.com/TesisEnel/AguaMariaSolution...,...,1.0,272,193,NaN,1.2,9.2,4075,1.0,1.0,37
2,3,2024-02,Adiel Luis García Rosa,1,adiel.garcia0422@gmail.com,https://github.com/SamyJp23/PeakPerformance.git,https://github.com/Adiel040/GymProApp.git,TesisEnel_PeakPerformance-samuelAntonio-ap1,TesisEnel_GymProApp-AdielGarcia-Ap2,https://github.com/TesisEnel/PeakPerformance-s...,...,1.0,375,546,NaN,1.5,0.8,5736,1.0,1.0,40
3,4,2024-03,Alaina Garcia Salazar,2,garciaalaina01@gmail.com,https://github.com/Reyx38/ReyAI_Transport.git,https://github.com/JeronyCruz/RecreArte.git,TesisEnel_ReyAI_Transport-Reyfil-Ap1,TesisEnel_RecreArte-JeronyCruz-Ap2,https://github.com/TesisEnel/ReyAI_Transport-R...,...,1.0,900,1038,NaN,2.8,12.3,11268,1.0,1.0,116
4,5,2023-01,Albert Luis Delgado Maria,1,bolshoi19booze@gmail.com,https://github.com/Rhazerpk/ProyectoFinal-AP1,https://github.com/Rhazerpk/MoonlightBarApp,TesisEnel_ProyectoFinal-AlbertRegalado-ap1,TesisEnel_MoonlightBarApp-AlbetDelgado-ap2,https://github.com/TesisEnel/ProyectoFinal-Alb...,...,1.0,99,172,NaN,1.5,10.7,2781,1.0,1.0,28


In [9]:
# Definir métricas principales por dimensión de calidad
# Las métricas están separadas por AP1 y AP2 en el dataset
base_metrics = {
    'Mantenibilidad': ['technical_debt', 'code_smells', 'duplicated_lines_density', 'cognitive_complexity'],
    'Fiabilidad': ['bugs', 'reliability_rating'],
    'Seguridad': ['vulnerabilities', 'security_rating']
}

# Métricas complementarias base
base_complementary = ['complexity', 'comment_lines_density', 'ncloc', 'security_hotspots', 'sqale_rating', 'open_issues']

# Crear listas completas con sufijos AP1 y AP2
quality_dimensions_ap1 = {}
quality_dimensions_ap2 = {}

for dimension, metrics in base_metrics.items():
    quality_dimensions_ap1[dimension] = [f"{metric}_AP1" for metric in metrics]
    quality_dimensions_ap2[dimension] = [f"{metric}_AP2" for metric in metrics]

complementary_metrics_ap1 = [f"{metric}_AP1" for metric in base_complementary]
complementary_metrics_ap2 = [f"{metric}_AP2" for metric in base_complementary]

# Todas las métricas de SonarCloud (AP1 y AP2)
all_metrics_ap1 = []
all_metrics_ap2 = []

for metrics_list in quality_dimensions_ap1.values():
    all_metrics_ap1.extend(metrics_list)
all_metrics_ap1.extend(complementary_metrics_ap1)

for metrics_list in quality_dimensions_ap2.values():
    all_metrics_ap2.extend(metrics_list)
all_metrics_ap2.extend(complementary_metrics_ap2)

# Métricas base (sin sufijo) para análisis
base_all_metrics = []
for metrics_list in base_metrics.values():
    base_all_metrics.extend(metrics_list)
base_all_metrics.extend(base_complementary)

print("=== MÉTRICAS PRINCIPALES POR DIMENSIÓN ===")
for dimension, metrics in base_metrics.items():
    print(f"\n🔍 {dimension}:")
    for metric in metrics:
        ap1_metric = f"{metric}_AP1"
        ap2_metric = f"{metric}_AP2"
        ap1_exists = ap1_metric in df.columns
        ap2_exists = ap2_metric in df.columns
        print(f"  {'✅' if ap1_exists and ap2_exists else '❌'} {metric}: AP1={ap1_exists}, AP2={ap2_exists}")

print(f"\n📊 Total de métricas base definidas: {len(base_all_metrics)}")
print(f"📊 Métricas AP1 disponibles: {len([m for m in all_metrics_ap1 if m in df.columns])}")
print(f"📊 Métricas AP2 disponibles: {len([m for m in all_metrics_ap2 if m in df.columns])}")

# Verificar que tenemos las columnas necesarias para análisis pareado
print(f"\n=== COLUMNAS CLAVE ===")
key_columns = ['Id', 'Estudiante', 'Semestre', 'Sexo']
for col in key_columns:
    print(f"{'✅' if col in df.columns else '❌'} {col}")

# Identificar estudiante como clave primaria
if 'Id' in df.columns:
    print(f"\n👥 Utilizando 'Id' como identificador de estudiante")
    student_id_col = 'Id'
elif 'Estudiante' in df.columns:
    print(f"\n👥 Utilizando 'Estudiante' como identificador de estudiante")
    student_id_col = 'Estudiante'
else:
    print(f"\n❌ No se encontró columna de identificación de estudiante")
    student_id_col = None

=== MÉTRICAS PRINCIPALES POR DIMENSIÓN ===

🔍 Mantenibilidad:
  ✅ technical_debt: AP1=True, AP2=True
  ✅ code_smells: AP1=True, AP2=True
  ✅ duplicated_lines_density: AP1=True, AP2=True
  ✅ cognitive_complexity: AP1=True, AP2=True

🔍 Fiabilidad:
  ✅ bugs: AP1=True, AP2=True
  ✅ reliability_rating: AP1=True, AP2=True

🔍 Seguridad:
  ✅ vulnerabilities: AP1=True, AP2=True
  ✅ security_rating: AP1=True, AP2=True

📊 Total de métricas base definidas: 14
📊 Métricas AP1 disponibles: 14
📊 Métricas AP2 disponibles: 14

=== COLUMNAS CLAVE ===
✅ Id
✅ Estudiante
✅ Semestre
✅ Sexo

👥 Utilizando 'Id' como identificador de estudiante


In [ ]:
# Estadísticas descriptivas por asignatura
print("=== ESTADÍSTICAS DESCRIPTIVAS POR ASIGNATURA ===")

# Filtrar métricas disponibles para cada asignatura
available_metrics_ap1 = [m for m in all_metrics_ap1 if m in df.columns]
available_metrics_ap2 = [m for m in all_metrics_ap2 if m in df.columns]

print(f"\n📚 ASIGNATURA AP1:")
print(f"👥 Número de estudiantes: {len(df)}")
print(f"📊 Métricas disponibles: {len(available_metrics_ap1)}")

if available_metrics_ap1:
    # Estadísticas de las métricas AP1
    stats_ap1 = df[available_metrics_ap1].describe()
    print("\n📈 Estadísticas descriptivas AP1:")
    display(stats_ap1.round(3))

print(f"\n? ASIGNATURA AP2:")
print(f"?👥 Número de estudiantes: {len(df)}")
print(f"📊 Métricas disponibles: {len(available_metrics_ap2)}")

if available_metrics_ap2:
    # Estadísticas de las métricas AP2
    stats_ap2 = df[available_metrics_ap2].describe()
    print("\n📈 Estadísticas descriptivas AP2:")
    display(stats_ap2.round(3))

# Comparación directa AP1 vs AP2 para métricas base
print("\n=== COMPARACIÓN DIRECTA AP1 vs AP2 ===")

comparison_data = []
for metric in base_all_metrics:
    ap1_col = f"{metric}_AP1"
    ap2_col = f"{metric}_AP2"
    
    if ap1_col in df.columns and ap2_col in df.columns:
        comparison_data.append({
            'Métrica': metric,
            'AP1_Media': df[ap1_col].mean(),
            'AP1_Mediana': df[ap1_col].median(),
            'AP1_Std': df[ap1_col].std(),
            'AP2_Media': df[ap2_col].mean(),
            'AP2_Mediana': df[ap2_col].median(),
            'AP2_Std': df[ap2_col].std(),
            'Diferencia_Media': df[ap2_col].mean() - df[ap1_col].mean(),
            'Cambio_Porcentual': ((df[ap2_col].mean() - df[ap1_col].mean()) / df[ap1_col].mean() * 100) if df[ap1_col].mean() != 0 else 0
        })

if comparison_data:
    comparison_df = pd.DataFrame(comparison_data)
    comparison_df = comparison_df.round(3)
    display(comparison_df)
    
    print(f"\n📊 Métricas comparables encontradas: {len(comparison_data)}")
else:
    print("\n❌ No se encontraron métricas comparables entre AP1 y AP2")

# 3. Data Preprocessing and Validation
## Preprocesamiento y Validación de Datos

In [ ]:
# Validar datos pareados
print("=== VALIDACIÓN DE DATOS PAREADOS ===")

if student_id_col:
    # En este formato, cada fila ya contiene datos de AP1 y AP2 para el mismo estudiante
    total_students = len(df)
    print(f"👥 Total de estudiantes en el dataset: {total_students}")
    
    # Verificar que tenemos datos para ambas asignaturas
    students_with_ap1 = df[available_metrics_ap1].notna().any(axis=1).sum()
    students_with_ap2 = df[available_metrics_ap2].notna().any(axis=1).sum()
    
    print(f"👥 Estudiantes con datos en AP1: {students_with_ap1}")
    print(f"👥 Estudiantes con datos en AP2: {students_with_ap2}")
    
    # Estudiantes con datos en ambas asignaturas
    has_ap1_data = df[available_metrics_ap1].notna().any(axis=1)
    has_ap2_data = df[available_metrics_ap2].notna().any(axis=1)
    paired_mask = has_ap1_data & has_ap2_data
    
    paired_students_count = paired_mask.sum()
    print(f"🔗 Estudiantes con datos pareados (AP1 y AP2): {paired_students_count}")
    
    # Filtrar datos para análisis pareado
    df_paired = df[paired_mask].copy()
    print(f"\n📊 Dataset pareado final: {df_paired.shape}")
    print(f"✅ Datos listos para análisis pre-post intervención")
    
    # Información adicional del dataset
    if 'Sexo' in df_paired.columns:
        print(f"\n👥 Distribución por género:")
        gender_dist = df_paired['Sexo'].value_counts()
        display(gender_dist)
    
    if 'Semestre' in df_paired.columns:
        print(f"\n? Distribución por semestre:")
        semester_dist = df_paired['Semestre'].value_counts()
        display(semester_dist)
        
else:
    print("❌ No se puede realizar validación sin columna de identificación de estudiante")
    df_paired = df.copy()  # Usar todo el dataset como fallback

In [ ]:
# Análisis de valores faltantes y outliers
print("=== ANÁLISIS DE VALORES FALTANTES ===")

# Analizar valores faltantes para AP1 y AP2 por separado
all_available_metrics = available_metrics_ap1 + available_metrics_ap2

missing_data = df_paired[all_available_metrics].isnull().sum()
missing_percentage = (missing_data / len(df_paired)) * 100

missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percentage.round(2)
})

# Separar por asignatura para mejor visualización
missing_ap1 = missing_df[missing_df.index.str.contains('_AP1')]
missing_ap2 = missing_df[missing_df.index.str.contains('_AP2')]

print("Valores faltantes AP1:")
display(missing_ap1[missing_ap1['Missing Count'] > 0])

print("Valores faltantes AP2:")
display(missing_ap2[missing_ap2['Missing Count'] > 0])

if missing_df['Missing Count'].sum() == 0:
    print("✅ No hay valores faltantes en las métricas principales")

# Detección de outliers usando IQR
print("\n=== DETECCIÓN DE OUTLIERS (IQR) ===")

def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return len(outliers), lower_bound, upper_bound

outlier_summary = []
for metric in all_available_metrics:
    if df_paired[metric].dtype in ['int64', 'float64']:
        n_outliers, lower, upper = detect_outliers_iqr(df_paired, metric)
        outlier_summary.append({
            'Metric': metric,
            'Outliers': n_outliers,
            'Lower Bound': lower,
            'Upper Bound': upper,
            'Outlier %': (n_outliers / len(df_paired)) * 100
        })

outlier_df = pd.DataFrame(outlier_summary)
print(f"📊 Resumen de outliers (primeras 10 métricas):")
display(outlier_df.head(10).round(3))

# Resumen general de outliers
high_outlier_metrics = outlier_df[outlier_df['Outlier %'] > 10]
if len(high_outlier_metrics) > 0:
    print(f"\n⚠️ Métricas con alto porcentaje de outliers (>10%):")
    display(high_outlier_metrics[['Metric', 'Outlier %']].round(1))
else:
    print(f"\n✅ No hay métricas con porcentajes extremos de outliers")

# 4. Normality Testing and Test Selection
## Pruebas de Normalidad y Selección de Tests Estadísticos

In [ ]:
def analyze_metric_evolution(df, base_metric, alpha=0.05):
    """
    Analiza la evolución de una métrica entre AP1 y AP2
    
    Parameters:
    df: DataFrame con datos pareados (cada fila contiene AP1 y AP2 del mismo estudiante)
    base_metric: nombre base de la métrica (sin sufijo _AP1 o _AP2)
    alpha: nivel de significancia (default 0.05)
    
    Returns:
    dict: resultados del análisis estadístico
    """
    
    # Construir nombres de columnas
    ap1_col = f"{base_metric}_AP1"
    ap2_col = f"{base_metric}_AP2"
    
    # Verificar que las columnas existan
    if ap1_col not in df.columns or ap2_col not in df.columns:
        return {'error': f'Columnas no encontradas: {ap1_col} o {ap2_col}'}
    
    # Extraer datos, eliminando valores nulos
    valid_mask = df[ap1_col].notna() & df[ap2_col].notna()
    ap1_data = df.loc[valid_mask, ap1_col]
    ap2_data = df.loc[valid_mask, ap2_col]
    
    # Verificar que tenemos datos suficientes
    if len(ap1_data) < 3:
        return {'error': f'Datos insuficientes: solo {len(ap1_data)} pares válidos'}
    
    # Calcular diferencias (AP2 - AP1)
    differences = ap2_data.values - ap1_data.values
    
    # Prueba de normalidad de las diferencias (Shapiro-Wilk)
    try:
        _, p_norm = shapiro(differences)
    except:
        p_norm = 0.01  # Asumir no normal si hay error
    
    # Seleccionar test apropiado basado en normalidad
    if p_norm > alpha:
        # Datos normales: T-test pareado
        try:
            stat, p_val = ttest_rel(ap1_data, ap2_data)
            test_used = 'T-test pareado'
        except:
            stat, p_val = wilcoxon(ap1_data, ap2_data, zero_method='zsplit')
            test_used = 'Wilcoxon signed-rank (fallback)'
    else:
        # Datos no normales: Wilcoxon signed-rank
        try:
            stat, p_val = wilcoxon(ap1_data, ap2_data, zero_method='zsplit')
            test_used = 'Wilcoxon signed-rank'
        except:
            stat, p_val = np.nan, np.nan
            test_used = 'Error en test'
    
    # Calcular tamaño del efecto (Cohen's d)
    pooled_std = np.sqrt(((ap1_data.std()**2 + ap2_data.std()**2) / 2))
    if pooled_std != 0:
        cohen_d = (ap2_data.mean() - ap1_data.mean()) / pooled_std
    else:
        cohen_d = 0
    
    # Interpretar tamaño del efecto
    if abs(cohen_d) < 0.2:
        effect_size = 'Pequeño'
    elif abs(cohen_d) < 0.5:
        effect_size = 'Mediano'
    elif abs(cohen_d) < 0.8:
        effect_size = 'Grande'
    else:
        effect_size = 'Muy grande'
    
    return {
        'metric': base_metric,
        'test': test_used,
        'statistic': stat,
        'p_value': p_val,
        'p_normality': p_norm,
        'is_significant': p_val < alpha if not np.isnan(p_val) else False,
        'cohen_d': cohen_d,
        'effect_size': effect_size,
        'ap1_mean': ap1_data.mean(),
        'ap1_std': ap1_data.std(),
        'ap2_mean': ap2_data.mean(),
        'ap2_std': ap2_data.std(),
        'improvement': ap2_data.mean() - ap1_data.mean(),
        'improvement_pct': ((ap2_data.mean() - ap1_data.mean()) / ap1_data.mean() * 100) if ap1_data.mean() != 0 else 0,
        'n_pairs': len(differences),
        'ap1_col': ap1_col,
        'ap2_col': ap2_col
    }

print("✅ Función analyze_metric_evolution actualizada para el nuevo formato")
print("🔬 Lista para análisis estadístico de métricas individuales")

# 5. Paired Statistical Testing for Individual Metrics
## Análisis Estadístico Pareado para Métricas Individuales

In [ ]:
# Análisis estadístico para todas las métricas disponibles
print("=== ANÁLISIS ESTADÍSTICO INDIVIDUAL DE MÉTRICAS ===")

results_individual = []

# Filtrar métricas base que tienen datos en ambas asignaturas
available_base_metrics = []
for metric in base_all_metrics:
    ap1_col = f"{metric}_AP1"
    ap2_col = f"{metric}_AP2"
    if ap1_col in df_paired.columns and ap2_col in df_paired.columns:
        available_base_metrics.append(metric)

print(f"📊 Métricas base disponibles para análisis: {len(available_base_metrics)}")

for metric in available_base_metrics:
    print(f"\n🔬 Analizando: {metric}")
    result = analyze_metric_evolution(df_paired, metric)
    
    if 'error' not in result:
        results_individual.append(result)
        
        # Mostrar resultados resumidos
        print(f"   📊 Test usado: {result['test']}")
        print(f"   📈 AP1 → AP2: {result['ap1_mean']:.3f} → {result['ap2_mean']:.3f}")
        print(f"   🔄 Cambio: {result['improvement']:.3f} ({result['improvement_pct']:.1f}%)")
        print(f"   🎯 p-valor: {result['p_value']:.4f}")
        print(f"   ⚡ Cohen's d: {result['cohen_d']:.3f} ({result['effect_size']})")
        print(f"   👥 Pares válidos: {result['n_pairs']}")
        print(f"   ✅ Significativo: {'Sí' if result['is_significant'] else 'No'}")
    else:
        print(f"   ❌ Error: {result['error']}")

# Crear DataFrame con resultados
if results_individual:
    results_df = pd.DataFrame(results_individual)
    print(f"\n📊 Análisis completado para {len(results_df)} métricas")
    print(f"✅ Métricas con mejora significativa: {sum(results_df['is_significant'])}")
    
    # Mostrar métricas con mejoras más significativas
    if sum(results_df['is_significant']) > 0:
        significant_results = results_df[results_df['is_significant']].sort_values('p_value')
        print(f"\n🌟 Top métricas con mejoras significativas:")
        for _, row in significant_results.head(5).iterrows():
            direction = "⬇️ Mejora" if row['improvement'] < 0 else "⬆️ Incremento"
            print(f"   • {row['metric']}: p={row['p_value']:.4f}, d={row['cohen_d']:.3f} {direction}")
else:
    print("❌ No se pudieron analizar las métricas")

In [ ]:
# Tabla resumen de resultados individuales
if results_individual:
    print("=== TABLA RESUMEN - ANÁLISIS INDIVIDUAL ===")
    
    summary_table = results_df[['metric', 'test', 'ap1_mean', 'ap2_mean', 'improvement', 
                               'improvement_pct', 'p_value', 'cohen_d', 'effect_size', 'is_significant']].copy()
    
    # Redondear valores numéricos
    summary_table['ap1_mean'] = summary_table['ap1_mean'].round(3)
    summary_table['ap2_mean'] = summary_table['ap2_mean'].round(3)
    summary_table['improvement'] = summary_table['improvement'].round(3)
    summary_table['improvement_pct'] = summary_table['improvement_pct'].round(1)
    summary_table['p_value'] = summary_table['p_value'].round(4)
    summary_table['cohen_d'] = summary_table['cohen_d'].round(3)
    
    # Renombrar columnas para mejor presentación
    summary_table.columns = ['Métrica', 'Test Estadístico', 'Media AP1', 'Media AP2', 
                           'Cambio Absoluto', 'Cambio %', 'p-valor', 'Cohen\'s d', 
                           'Tamaño Efecto', 'Significativo']
    
    display(summary_table)
    
    # Estadísticas generales
    print(f"\n=== RESUMEN GENERAL ===")
    print(f"📊 Total de métricas analizadas: {len(results_df)}")
    print(f"✅ Métricas con mejora significativa (p < 0.05): {sum(results_df['is_significant'])}")
    print(f"📈 Métricas con tamaño de efecto grande (|d| > 0.8): {sum(abs(results_df['cohen_d']) > 0.8)}")
    print(f"📉 Métricas con mejora (cambio negativo en métricas 'malas'): {sum(results_df['improvement'] < 0)}")
    print(f"🔬 Tests paramétricos utilizados: {sum(results_df['test'].str.contains('T-test'))}")
    print(f"🔬 Tests no paramétricos utilizados: {sum(results_df['test'].str.contains('Wilcoxon'))}")

# 6. Quality Dimensions Analysis
## Análisis por Dimensiones de Calidad

In [ ]:
# Análisis por dimensiones de calidad
print("=== ANÁLISIS POR DIMENSIONES DE CALIDAD ===")

dimension_results = {}

for dimension, base_metrics_list in base_metrics.items():
    print(f"\n🏗️ DIMENSIÓN: {dimension.upper()}")
    print("=" * 50)
    
    dimension_data = []
    available_dimension_metrics = [m for m in base_metrics_list if m in available_base_metrics]
    
    for metric in available_dimension_metrics:
        # Buscar resultado del análisis individual
        metric_result = next((r for r in results_individual if r['metric'] == metric), None)
        
        if metric_result:
            dimension_data.append(metric_result)
            
            print(f"\n📊 {metric}:")
            print(f"   📈 Cambio: {metric_result['ap1_mean']:.3f} → {metric_result['ap2_mean']:.3f}")
            print(f"   🔄 Mejora: {metric_result['improvement']:.3f} ({metric_result['improvement_pct']:.1f}%)")
            print(f"   🎯 p-valor: {metric_result['p_value']:.4f}")
            print(f"   ⚡ Cohen's d: {metric_result['cohen_d']:.3f}")
            print(f"   👥 Pares: {metric_result['n_pairs']}")
            print(f"   ✅ Significativo: {'Sí' if metric_result['is_significant'] else 'No'}")
    
    # Resumen por dimensión
    if dimension_data:
        significant_count = sum(1 for d in dimension_data if d['is_significant'])
        avg_effect_size = np.mean([d['cohen_d'] for d in dimension_data])
        avg_improvement = np.mean([d['improvement_pct'] for d in dimension_data])
        
        dimension_results[dimension] = {
            'metrics_count': len(dimension_data),
            'significant_count': significant_count,
            'significant_pct': (significant_count / len(dimension_data)) * 100,
            'avg_effect_size': avg_effect_size,
            'avg_improvement_pct': avg_improvement,
            'metrics_data': dimension_data
        }
        
        print(f"\n📋 RESUMEN {dimension}:")
        print(f"   📊 Métricas analizadas: {len(dimension_data)}")
        print(f"   ✅ Mejoras significativas: {significant_count}/{len(dimension_data)} ({significant_count/len(dimension_data)*100:.1f}%)")
        print(f"   ⚡ Tamaño de efecto promedio: {avg_effect_size:.3f}")
        print(f"   📈 Cambio promedio: {avg_improvement:.1f}%")
    else:
        print(f"   ❌ No hay métricas disponibles para esta dimensión")

# Resumen general por dimensiones
print(f"\n=== RESUMEN GENERAL POR DIMENSIONES ===")
for dimension, summary in dimension_results.items():
    improvement_direction = "⬇️" if summary['avg_improvement_pct'] < 0 else "⬆️"
    print(f"🏗️ {dimension}: {summary['significant_count']}/{summary['metrics_count']} significativas ({summary['significant_pct']:.1f}%)")
    print(f"   ⚡ Efecto promedio: {summary['avg_effect_size']:.3f}")
    print(f"   📈 Cambio promedio: {improvement_direction} {abs(summary['avg_improvement_pct']):.1f}%")

# 7. Multiple Comparisons Correction
## Corrección por Comparaciones Múltiples

In [ ]:
# Corrección por comparaciones múltiples usando FDR (Benjamini-Hochberg)
print("=== CORRECCIÓN POR COMPARACIONES MÚLTIPLES (FDR) ===")

if results_individual:
    # Extraer p-valores
    p_values = [r['p_value'] for r in results_individual]
    metric_names = [r['metric'] for r in results_individual]
    
    # Aplicar corrección FDR (Benjamini-Hochberg)
    rejected, corrected_p, alpha_sidak, alpha_bonf = multipletests(
        p_values, alpha=0.05, method='fdr_bh'
    )
    
    # Crear DataFrame con resultados corregidos
    correction_results = pd.DataFrame({
        'Métrica': metric_names,
        'p-valor_original': p_values,
        'p-valor_corregido_FDR': corrected_p,
        'Significativo_original': [p < 0.05 for p in p_values],
        'Significativo_FDR': rejected,
        'Cohen_d': [r['cohen_d'] for r in results_individual],
        'Cambio_absoluto': [r['improvement'] for r in results_individual],
        'Cambio_porcentual': [r['improvement_pct'] for r in results_individual]
    })
    
    # Redondear valores
    correction_results['p-valor_original'] = correction_results['p-valor_original'].round(4)
    correction_results['p-valor_corregido_FDR'] = correction_results['p-valor_corregido_FDR'].round(4)
    correction_results['Cohen_d'] = correction_results['Cohen_d'].round(3)
    correction_results['Cambio_absoluto'] = correction_results['Cambio_absoluto'].round(3)
    correction_results['Cambio_porcentual'] = correction_results['Cambio_porcentual'].round(1)
    
    # Ordenar por p-valor corregido
    correction_results = correction_results.sort_values('p-valor_corregido_FDR')
    
    display(correction_results)
    
    # Resumen de corrección
    original_significant = sum(correction_results['Significativo_original'])
    fdr_significant = sum(correction_results['Significativo_FDR'])
    
    print(f"\n=== IMPACTO DE LA CORRECCIÓN FDR ===")
    print(f"✅ Significativas sin corrección (α = 0.05): {original_significant}/{len(p_values)} ({original_significant/len(p_values)*100:.1f}%)")
    print(f"✅ Significativas con corrección FDR: {fdr_significant}/{len(p_values)} ({fdr_significant/len(p_values)*100:.1f}%)")
    print(f"📉 Diferencia: {original_significant - fdr_significant} métricas")
    
    if fdr_significant > 0:
        print(f"\n🎯 MÉTRICAS SIGNIFICATIVAS DESPUÉS DE CORRECCIÓN FDR:")
        significant_fdr = correction_results[correction_results['Significativo_FDR']]
        for _, row in significant_fdr.iterrows():
            print(f"   ✅ {row['Métrica']}: p-FDR = {row['p-valor_corregido_FDR']:.4f}, d = {row['Cohen_d']:.3f}")
    else:
        print("⚠️ Ninguna métrica permanece significativa después de la corrección FDR")
else:
    print("❌ No hay resultados para corregir")

# 8. Effect Size Calculations
## Análisis de Tamaño del Efecto

In [ ]:
# Análisis detallado del tamaño del efecto
print("=== ANÁLISIS DE TAMAÑO DEL EFECTO (COHEN'S D) ===")

if results_individual:
    # Crear DataFrame para análisis de tamaño del efecto
    effect_size_df = pd.DataFrame({
        'Métrica': [r['metric'] for r in results_individual],
        'Cohen_d': [r['cohen_d'] for r in results_individual],
        'Efecto_categoría': [r['effect_size'] for r in results_individual],
        'p_valor': [r['p_value'] for r in results_individual],
        'Significativo': [r['is_significant'] for r in results_individual],
        'Cambio_absoluto': [r['improvement'] for r in results_individual],
        'Cambio_porcentual': [r['improvement_pct'] for r in results_individual]
    })
    
    # Agregar valor absoluto de Cohen's d para ordenamiento
    effect_size_df['Cohen_d_abs'] = abs(effect_size_df['Cohen_d'])
    
    # Ordenar por tamaño del efecto (valor absoluto)
    effect_size_df = effect_size_df.sort_values('Cohen_d_abs', ascending=False)
    
    print("📊 RANKING DE MÉTRICAS POR TAMAÑO DEL EFECTO:")
    display(effect_size_df[['Métrica', 'Cohen_d', 'Efecto_categoría', 'p_valor', 'Significativo']].round(3))
    
    # Análisis por categoría de efecto
    print(f"\n=== DISTRIBUCIÓN POR TAMAÑO DEL EFECTO ===")
    effect_counts = effect_size_df['Efecto_categoría'].value_counts()
    
    for category in ['Muy grande', 'Grande', 'Mediano', 'Pequeño']:
        count = effect_counts.get(category, 0)
        percentage = (count / len(effect_size_df)) * 100
        print(f"⚡ {category}: {count} métricas ({percentage:.1f}%)")
    
    # Métricas con efecto grande o muy grande
    large_effects = effect_size_df[effect_size_df['Cohen_d_abs'] >= 0.8]
    if len(large_effects) > 0:
        print(f"\n🎯 MÉTRICAS CON EFECTO GRANDE O MUY GRANDE (|d| ≥ 0.8):")
        for _, row in large_effects.iterrows():
            direction = "Mejora" if row['Cohen_d'] < 0 else "Empeoramiento"
            print(f"   ⚡ {row['Métrica']}: d = {row['Cohen_d']:.3f} ({direction})")
    
    # Métricas con efecto mediano o mayor Y significativas
    meaningful_changes = effect_size_df[
        (effect_size_df['Cohen_d_abs'] >= 0.5) & 
        (effect_size_df['Significativo'] == True)
    ]
    
    if len(meaningful_changes) > 0:
        print(f"\n✨ CAMBIOS SIGNIFICATIVOS Y SUSTANCIALES (|d| ≥ 0.5 y p < 0.05):")
        for _, row in meaningful_changes.iterrows():
            direction = "Mejora" if row['Cohen_d'] < 0 else "Empeoramiento"
            print(f"   🌟 {row['Métrica']}: d = {row['Cohen_d']:.3f}, p = {row['p_valor']:.4f} ({direction})")
    else:
        print(f"\n⚠️ No se encontraron cambios significativos y sustanciales")
    
    # Resumen estadístico del tamaño del efecto
    print(f"\n=== ESTADÍSTICAS DEL TAMAÑO DEL EFECTO ===")
    print(f"📊 Cohen's d promedio: {effect_size_df['Cohen_d'].mean():.3f}")
    print(f"📊 Cohen's d mediana: {effect_size_df['Cohen_d'].median():.3f}")
    print(f"📊 Cohen's d rango: [{effect_size_df['Cohen_d'].min():.3f}, {effect_size_df['Cohen_d'].max():.3f}]")
    print(f"📊 |Cohen's d| promedio: {effect_size_df['Cohen_d_abs'].mean():.3f}")
else:
    print("❌ No hay resultados para analizar el tamaño del efecto")

# 9. Statistical Visualizations
## Visualizaciones Estadísticas

In [ ]:
# Box plots comparativos AP1 vs AP2 para métricas principales
main_base_metrics = ['technical_debt', 'code_smells', 'bugs', 'vulnerabilities', 
                     'duplicated_lines_density', 'cognitive_complexity', 'reliability_rating', 'security_rating']

# Filtrar métricas disponibles
available_main_metrics = [m for m in main_base_metrics if m in available_base_metrics]

if available_main_metrics:
    # Configurar subplots
    n_metrics = len(available_main_metrics)
    cols = 4
    rows = (n_metrics + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 5*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, metric in enumerate(available_main_metrics):
        row = i // cols
        col = i % cols
        ax = axes[row, col]
        
        # Preparar datos para boxplot
        ap1_col = f"{metric}_AP1"
        ap2_col = f"{metric}_AP2"
        
        # Crear datos en formato largo para seaborn
        ap1_data = df_paired[ap1_col].dropna()
        ap2_data = df_paired[ap2_col].dropna()
        
        plot_data = pd.DataFrame({
            'Valores': pd.concat([ap1_data, ap2_data]),
            'Asignatura': ['AP1'] * len(ap1_data) + ['AP2'] * len(ap2_data)
        })
        
        # Crear box plot
        sns.boxplot(data=plot_data, x='Asignatura', y='Valores', ax=ax)
        ax.set_title(f'{metric}')
        ax.set_xlabel('Asignatura')
        ax.set_ylabel('Valor')
        
        # Agregar información estadística
        result = next((r for r in results_individual if r['metric'] == metric), None)
        if result:
            significance = "***" if result['p_value'] < 0.001 else "**" if result['p_value'] < 0.01 else "*" if result['p_value'] < 0.05 else "ns"
            ax.text(0.5, 0.95, f"p = {result['p_value']:.3f} {significance}\nd = {result['cohen_d']:.3f}", 
                   transform=ax.transAxes, ha='center', va='top', 
                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Ocultar subplots vacíos
    for i in range(len(available_main_metrics), rows * cols):
        row = i // cols
        col = i % cols
        axes[row, col].set_visible(False)
    
    plt.suptitle('Comparación de Métricas de Calidad: AP1 vs AP2', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()
else:
    print("❌ No hay métricas principales disponibles para visualizar")

In [ ]:
# Heatmap de correlaciones entre métricas
if len(available_main_metrics) > 1:
    # Crear dataset combinado para correlaciones
    correlation_data = pd.DataFrame()
    
    # Agregar datos de AP1 y AP2 para cada métrica
    for metric in available_main_metrics:
        ap1_col = f"{metric}_AP1"
        ap2_col = f"{metric}_AP2"
        
        if ap1_col in df_paired.columns and ap2_col in df_paired.columns:
            # Combinar datos de ambas asignaturas
            combined_values = pd.concat([df_paired[ap1_col].dropna(), df_paired[ap2_col].dropna()])
            correlation_data[metric] = combined_values.reset_index(drop=True)
    
    # Asegurar que todas las columnas tengan la misma longitud
    min_length = min([len(correlation_data[col].dropna()) for col in correlation_data.columns])
    for col in correlation_data.columns:
        correlation_data[col] = correlation_data[col].head(min_length)
    
    if len(correlation_data.columns) > 1:
        plt.figure(figsize=(12, 10))
        
        # Calcular matriz de correlación
        correlation_matrix = correlation_data.corr()
        
        # Crear heatmap
        mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
        sns.heatmap(correlation_matrix, 
                    annot=True, 
                    cmap='coolwarm', 
                    center=0,
                    mask=mask,
                    square=True,
                    fmt='.3f',
                    cbar_kws={"shrink": .8})
        
        plt.title('Matriz de Correlaciones - Métricas de Calidad de Código\n(Datos combinados AP1 y AP2)', fontsize=14, pad=20)
        plt.tight_layout()
        plt.show()
        
        # Mostrar correlaciones más fuertes
        print("=== CORRELACIONES MÁS FUERTES ===")
        corr_pairs = []
        for i in range(len(correlation_matrix.columns)):
            for j in range(i+1, len(correlation_matrix.columns)):
                corr_value = correlation_matrix.iloc[i, j]
                if abs(corr_value) > 0.5:  # Correlaciones moderadas a fuertes
                    corr_pairs.append({
                        'Métrica 1': correlation_matrix.columns[i],
                        'Métrica 2': correlation_matrix.columns[j],
                        'Correlación': corr_value
                    })
        
        if corr_pairs:
            corr_df = pd.DataFrame(corr_pairs)
            corr_df = corr_df.sort_values('Correlación', key=abs, ascending=False)
            display(corr_df.round(3))
        else:
            print("No se encontraron correlaciones fuertes (|r| > 0.5)")
    else:
        print("❌ Insuficientes métricas válidas para análisis de correlación")
else:
    print("❌ Insuficientes métricas para análisis de correlación")

In [ ]:
# Gráficos de evolución individual por estudiante (muestra)
# Mostrar evolución para las 3 métricas más significativas
if results_individual:
    # Obtener las 3 métricas con menor p-valor
    top_significant = sorted(results_individual, key=lambda x: x['p_value'])[:3]
    
    if len(top_significant) > 0:
        n_plots = len(top_significant)
        fig, axes = plt.subplots(1, n_plots, figsize=(6*n_plots, 6))
        if n_plots == 1:
            axes = [axes]
        
        for i, result in enumerate(top_significant):
            metric = result['metric']
            ap1_col = result['ap1_col']
            ap2_col = result['ap2_col']
            ax = axes[i]
            
            # Obtener datos válidos (sin valores nulos)
            valid_mask = df_paired[ap1_col].notna() & df_paired[ap2_col].notna()
            valid_data = df_paired[valid_mask]
            
            ap1_values = valid_data[ap1_col]
            ap2_values = valid_data[ap2_col]
            
            # Tomar una muestra de estudiantes para visualización clara
            sample_size = min(20, len(ap1_values))
            if len(ap1_values) > sample_size:
                sample_indices = np.random.choice(len(ap1_values), sample_size, replace=False)
                ap1_sample = ap1_values.iloc[sample_indices]
                ap2_sample = ap2_values.iloc[sample_indices]
            else:
                ap1_sample = ap1_values
                ap2_sample = ap2_values
            
            # Crear líneas conectando AP1 y AP2 para cada estudiante
            for j in range(len(ap1_sample)):
                ax.plot([1, 2], [ap1_sample.iloc[j], ap2_sample.iloc[j]], 
                       'o-', alpha=0.6, linewidth=1, markersize=4, color='lightblue')
            
            # Medias
            ax.plot([1, 2], [ap1_values.mean(), ap2_values.mean()], 
                   'ro-', linewidth=3, markersize=8, label='Media', color='red')
            
            # Medianas
            ax.plot([1, 2], [ap1_values.median(), ap2_values.median()], 
                   'go-', linewidth=2, markersize=6, label='Mediana', color='green')
            
            ax.set_xlim(0.8, 2.2)
            ax.set_xticks([1, 2])
            ax.set_xticklabels(['AP1', 'AP2'])
            ax.set_ylabel(metric)
            ax.set_title(f'{metric}\n(p = {result["p_value"]:.4f}, d = {result["cohen_d"]:.3f})')
            ax.grid(True, alpha=0.3)
            ax.legend()
        
        plt.suptitle(f'Evolución Individual por Estudiante (Muestra de {sample_size})', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # Mostrar estadísticas de las métricas más significativas
        print(f"\n📊 ESTADÍSTICAS DE LAS MÉTRICAS MÁS SIGNIFICATIVAS:")
        for result in top_significant:
            print(f"\n🎯 {result['metric']}:")
            print(f"   • p-valor: {result['p_value']:.4f}")
            print(f"   • Cohen's d: {result['cohen_d']:.3f} ({result['effect_size']})")
            print(f"   • Cambio: {result['improvement']:.3f} ({result['improvement_pct']:.1f}%)")
            print(f"   • Pares analizados: {result['n_pairs']}")
    else:
        print("❌ No hay resultados significativos para mostrar")
else:
    print("❌ No hay resultados para mostrar evolución individual")

# 10. Results Summary and Interpretation
## Resumen de Resultados e Interpretación

In [ ]:
# Tabla resumen final con todas las métricas y resultados
print("=== TABLA RESUMEN FINAL - ANÁLISIS ESTADÍSTICO COMPLETO ===")

if results_individual:
    # Crear tabla resumen final
    final_summary = pd.DataFrame({
        'Métrica': [r['metric'] for r in results_individual],
        'Dimensión': [next((dim for dim, metrics in quality_dimensions.items() if r['metric'] in metrics), 'Complementaria') 
                     for r in results_individual],
        'AP1_Media': [r['ap1_mean'] for r in results_individual],
        'AP1_Std': [r['ap1_std'] for r in results_individual],
        'AP2_Media': [r['ap2_mean'] for r in results_individual],
        'AP2_Std': [r['ap2_std'] for r in results_individual],
        'Cambio_Absoluto': [r['improvement'] for r in results_individual],
        'Cambio_Porcentual': [r['improvement_pct'] for r in results_individual],
        'Test_Estadístico': [r['test'] for r in results_individual],
        'p_valor': [r['p_value'] for r in results_individual],
        'Cohen_d': [r['cohen_d'] for r in results_individual],
        'Tamaño_Efecto': [r['effect_size'] for r in results_individual],
        'Significativo': [r['is_significant'] for r in results_individual]
    })
    
    # Agregar información de corrección FDR si está disponible
    if 'correction_results' in locals():
        fdr_significant = correction_results.set_index('Métrica')['Significativo_FDR'].to_dict()
        final_summary['Significativo_FDR'] = final_summary['Métrica'].map(fdr_significant)
    
    # Redondear valores numéricos
    numeric_cols = ['AP1_Media', 'AP1_Std', 'AP2_Media', 'AP2_Std', 'Cambio_Absoluto', 'Cambio_Porcentual', 'p_valor', 'Cohen_d']
    final_summary[numeric_cols] = final_summary[numeric_cols].round(3)
    
    # Ordenar por dimensión y luego por p-valor
    final_summary = final_summary.sort_values(['Dimensión', 'p_valor'])
    
    display(final_summary)
    
    # Exportar tabla para uso posterior
    final_summary.to_csv('resultados_analisis_estadistico.csv', index=False)
    print("\n💾 Tabla guardada como 'resultados_analisis_estadistico.csv'")
else:
    print("❌ No hay resultados para mostrar en la tabla resumen final")

In [ ]:
# Interpretación y conclusiones finales
print("=== INTERPRETACIÓN Y CONCLUSIONES ===")

if results_individual:
    # Calcular estadísticas generales
    total_metrics = len(results_individual)
    significant_metrics = sum(r['is_significant'] for r in results_individual)
    large_effects = sum(abs(r['cohen_d']) >= 0.8 for r in results_individual)
    medium_effects = sum(0.5 <= abs(r['cohen_d']) < 0.8 for r in results_individual)
    
    print(f"📊 RESUMEN ESTADÍSTICO GENERAL:")
    print(f"   • Total de métricas analizadas: {total_metrics}")
    print(f"   • Métricas con diferencias significativas (p < 0.05): {significant_metrics} ({significant_metrics/total_metrics*100:.1f}%)")
    print(f"   • Métricas con efecto grande (|d| ≥ 0.8): {large_effects} ({large_effects/total_metrics*100:.1f}%)")
    print(f"   • Métricas con efecto mediano (0.5 ≤ |d| < 0.8): {medium_effects} ({medium_effects/total_metrics*100:.1f}%)")
    
    # Análisis por dimensión
    print(f"\n🏗️ ANÁLISIS POR DIMENSIÓN DE CALIDAD:")
    for dimension, summary in dimension_results.items():
        print(f"   • {dimension}: {summary['significant_count']}/{summary['metrics_count']} significativas ({summary['significant_pct']:.1f}%)")
        print(f"     - Tamaño de efecto promedio: {summary['avg_effect_size']:.3f}")
    
    # Respuesta a la hipótesis de investigación
    print(f"\n🎯 RESPUESTA A LA HIPÓTESIS DE INVESTIGACIÓN:")
    if significant_metrics > 0:
        print(f"   ✅ RECHAZAMOS H₀: Existe evidencia estadística de diferencias significativas")
        print(f"   ✅ ACEPTAMOS H₁: Existe mejora estadísticamente significativa en {significant_metrics} métricas")
        print(f"   📈 La intervención pedagógica (taller 'Código Limpio') mostró efectos positivos")
    else:
        print(f"   ❌ NO RECHAZAMOS H₀: No hay evidencia estadística suficiente de diferencias")
        print(f"   ❌ NO ACEPTAMOS H₁: No se detectó mejora estadísticamente significativa")
    
    # Interpretación práctica
    print(f"\n💡 INTERPRETACIÓN PRÁCTICA:")
    
    # Métricas que mejoraron significativamente
    improved_metrics = [r for r in results_individual if r['is_significant'] and r['improvement'] < 0]
    if improved_metrics:
        print(f"   🌟 Métricas que mejoraron significativamente (valores menores son mejores):")
        for result in improved_metrics:
            print(f"      - {result['metric']}: {result['improvement_pct']:.1f}% de mejora")
    
    # Métricas que empeoraron significativamente
    worsened_metrics = [r for r in results_individual if r['is_significant'] and r['improvement'] > 0]
    if worsened_metrics:
        print(f"   ⚠️ Métricas que empeoraron significativamente:")
        for result in worsened_metrics:
            print(f"      - {result['metric']}: {result['improvement_pct']:.1f}% de empeoramiento")
    
    # Limitaciones del estudio
    print(f"\n⚠️ LIMITACIONES DEL ANÁLISIS:")
    print(f"   • Diseño pre-post sin grupo control")
    print(f"   • Posibles variables confusoras no controladas")
    print(f"   • Efectos del aprendizaje natural a lo largo del tiempo")
    print(f"   • Diferencias en la complejidad de los proyectos entre asignaturas")
    
    # Recomendaciones para investigación futura
    print(f"\n🔮 RECOMENDACIONES PARA INVESTIGACIÓN FUTURA:")
    print(f"   • Incluir grupo control sin intervención")
    print(f"   • Análisis longitudinal con más puntos de medición")
    print(f"   • Análisis cualitativo complementario")
    print(f"   • Considerar variables moderadoras (experiencia previa, motivación)")
    
else:
    print("❌ No se pueden generar conclusiones sin resultados válidos")

print(f"\n🎊 ANÁLISIS ESTADÍSTICO COMPLETADO")
print(f"📋 Todos los resultados están disponibles en las secciones anteriores")
print(f"💾 Tablas exportadas para uso posterior en la tesis")